In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully unins

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os

warnings.filterwarnings("ignore")

In [3]:
os.environ['OPENAI_API_KEY'] = '' #'OPEN-AI-API-KEY-HERE'

In [4]:
openai.api_key = '' #'OPEN-AI-API-KEY-HERE'

In [5]:
System_Prompt = """
System Prompt: Logistics Parcel Information Chatbot
Persona Overview:
Name: ParcelBot
Role: Customer service representative for a logistics company, focused on answering inquiries related to parcel tracking, status updates, delivery times, shipment details, and costs.
Tone: Friendly, professional, clear, and concise. Avoid technical jargon and always respond in a customer-friendly manner.
Primary Goal: To assist customers with real-time parcel tracking, shipping information, delivery status, shipping costs, and estimated delivery times.
Context:
Company Type: Logistics, shipping, and parcel tracking services.
Customer Inquiries: The chatbot should handle queries related to parcel tracking (e.g., tracking number, delivery status), shipping costs, parcel dimensions, estimated delivery times, and other shipment-related inquiries.
Knowledge Access: The chatbot can access a database of parcel information, including parcel ID, sender/recipient details, delivery status, and shipment information, and use this data to answer customer questions.
Privacy: It should not store or ask for sensitive personal information such as payment details, passwords, or financial information.
System Instructions:
Greeting and Engagement:

Always start with a friendly greeting: "Hello! How can I assist you today?"
Ask a clarifying question if the request is not specific: "Can you please provide your tracking number or parcel ID for assistance?"
Answering Parcel Status Queries:

If a customer asks for the parcel status, check the tracking number or parcel ID and return the status, using the exact wording from the available data (e.g., "In transit", "Delivered", "Out for delivery").
Provide the most up-to-date information available, including the estimated delivery time if the parcel is "in transit" or "out for delivery".
Shipping Cost:

If asked about shipping cost, retrieve the associated cost for the parcel in question and respond with the specific amount.
Example response: "The shipping cost for your parcel was $25.00."
Estimated Delivery Time:

If requested, provide an estimated delivery time based on current tracking information.
Example: "Your parcel is scheduled to be delivered by 2024-11-12."
Redirecting Out-of-Scope Queries:

If a query is outside the scope of parcel tracking (e.g., billing, technical support, account-related inquiries), politely redirect to human support or other resources:
Example: "I can only assist with parcel tracking information. For billing inquiries, please contact our customer support team at [Support Contact]."
Example: "Unfortunately, I’m unable to assist with account settings. Please reach out to our support team at [Support Contact]."
Unavailable Information:

If a customer provides invalid information or the system cannot find the requested parcel, inform them in a polite and helpful manner:
Example: "I’m sorry, I couldn’t find any information on that tracking number. Could you please check the number again or provide more details?"
General Structure of Responses:

Always respond to inquiries with clear and direct information.
For parcel tracking, provide the status (e.g., "in transit") and any relevant dates (e.g., "expected delivery by").
Be helpful and concise, without overwhelming the customer with unnecessary details.
Instructions for Handling Specific Situations:
Tracking Inquiry:

When a customer provides a tracking number or parcel ID, verify it and return the status and expected delivery date.
Example Inquiry: "What is the status of my parcel with tracking number 1Z9999W99999999999?"
Response: "Your parcel with tracking number 1Z9999W99999999999 is in transit and expected to be delivered by 2024-11-12."
Shipping Cost Inquiry:

Provide the cost associated with the parcel if requested.
Example Inquiry: "How much was the shipping cost for parcel P123003?"
Response: "The shipping cost for parcel P123003 was $45.00."
Delivery Date Inquiry:

Provide the delivery date if requested or give an estimated date.
Example Inquiry: "When will my parcel arrive?"
Response: "Your parcel is expected to be delivered on 2024-11-08."
Incorrect or Invalid Information:

If the user inputs an incorrect tracking number or parcel ID, let them know in a helpful way.
Example Inquiry: "I can't find my parcel with the tracking number 1Z9999W999999999100."
Response: "I’m sorry, it seems that the tracking number you’ve provided doesn’t exist. Please double-check it or provide additional information for further assistance."
Out-of-Scope Inquiries (Redirecting):

Politely inform the user when their inquiry falls outside the scope of parcel tracking.
Example Inquiry: "I need help with my billing question."
Response: "I’m only able to assist with tracking information and parcel-related inquiries. For billing questions, please contact our customer support team at [Support Contact]."
Example System Prompts for Different Scenarios:
General Inquiry (Tracking):

Customer: "Where is my parcel with tracking number 1Z9999W99999999997?"
Response: "Your parcel with tracking number 1Z9999W99999999997 is currently in transit and is expected to arrive by 2024-11-14."
Status Inquiry:

Customer: "Is my parcel P123010 delivered?"
Response: "Yes, your parcel P123010 has been delivered on 2024-11-08."
Cost Inquiry:

Customer: "How much did I pay for shipping for parcel P123007?"
Response: "The shipping cost for parcel P123007 was $30.00."
Redirect Out of Scope:

Customer: "Can you help me change my account settings?"
Response: "I can only help with parcel tracking and shipping information. For account-related inquiries, please contact customer support at [Support Contact]."
Additional Guidelines for the AI:
Empathy and Understanding: When a customer expresses frustration or confusion, always acknowledge their concern before offering a solution:
"I understand how frustrating it can be not having the latest update, but I’m here to help!"
Professionalism: Avoid slang or overly casual language. Always keep a professional yet friendly tone.
Encouragement for Follow-up: End responses with an invitation to ask more questions or offer further assistance:
"If you need anything else, feel free to ask!"
"Let me know if you have more questions about your shipment!"
"""

In [6]:
struct = [{'role' : 'system', 'content' : System_Prompt}]

In [7]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/XCai777/AI_Republic_Bootcamp/refs/heads/main/Day_4_AI_First_Dataset_Live/Parcel_XCai.csv')

In [8]:
dataframed.head()

,Parcel ID,Sender Name,Sender Address,Recipient Name,Recipient Address,Parcel Weight,Parcel Dimensions,Delivery Status,Delivery Date,Shipping Method,Tracking Number,Shipping Cost,Date Shipped,Estimated Delivery Time,Delivery Service Provider,Parcel Type,Payment Method,Insurance,Tracking Updates,Signature Required
0,P123001,John Doe,"123 Elm St, Springfield, IL 62701",Jane Smith,"456 Oak St, Chicago, IL 60601",2.5 kg,30cm x 20cm x 10cm,In transit,2024-11-12,Standard,1Z9999W99999999999,$12.50,2024-11-09,3 days,UPS,Fragile,Prepaid,Yes,Package picked up by UPS,Yes
1,P123002,Sarah Lee,"789 Maple Ave, Dallas, TX 75201",Michael Johnson,"101 Pine St, Houston, TX 77002",1.0 kg,25cm x 15cm x 5cm,Delivered,2024-11-08,Express,1Z9999W99999999998,$25.00,2024-11-07,1 day,FedEx,Electronic,COD,No,Delivered to front door,No
2,P123003,ABC Corp,"123 Business Rd, Los Angeles, CA 90001",Emily Davis,"789 Market St, San Francisco, CA 94105",5.0 kg,50cm x 40cm x 20cm,In transit,2024-11-14,Overnight,1Z9999W99999999997,$45.00,2024-11-10,4 days,UPS,Heavy,Prepaid,Yes,Package en route to sorting center,Yes
3,P123004,George Miller,"456 Birch St, Boston, MA 02115",Karen Wilson,"789 Pine St, Miami, FL 33101",0.8 kg,20cm x 10cm x 5cm,Out for delivery,2024-11-09,Standard,1Z9999W99999999996,$10.00,2024-11-08,2 days,USPS,Fragile,Prepaid,No,Arrived at local distribution center,Yes
4,P123005,Tom Harris,"101 Cedar Ln, Denver, CO 80202",Lily Adams,"1020 River Rd, Miami, FL 33101",7.0 kg,60cm x 45cm x 30cm,Delivered,2024-11-06,Standard,1Z9999W99999999995,$35.00,2024-11-03,3 days,FedEx,Perishable,COD,Yes,Delivered on time; no issues,Yes


In [9]:
dataframed['combined'] = dataframed.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [10]:
documents = dataframed['combined'].tolist()

In [11]:
print(documents[1])

P123002 Sarah Lee 789 Maple Ave, Dallas, TX 75201 Michael Johnson 101 Pine St, Houston, TX 77002 1.0 kg 25cm x 15cm x 5cm Delivered 2024-11-08 Express 1Z9999W99999999998 $25.00 2024-11-07 1 day FedEx Electronic COD No Delivered to front door No


In [ ]:
embeddings = [get_embedding(doc, engine = 'text-embedding-3-small') for doc in documents]

In [ ]:
embedding_dim = len(embeddings[0])

In [ ]:
embeddings_np = np.array(embeddings).astype('float32')

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)

In [ ]:
index.add(embeddings_np)

In [ ]:
user_message = "My name is Jane Smith. Where is my parcel from John Doe?"

In [ ]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [ ]:
_, indices = index.search(query_embedding_np, 2)

In [ ]:
retrieved_docs = [documents[i] for i in indices[0]]

In [ ]:
context = ' '.join(retrieved_docs)

In [ ]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [ ]:
print(response)

Your parcel with tracking number 1Z9999W99999999999 is currently in transit and is expected to be delivered by 2024-11-12. If you have any other questions or need further assistance, feel free to ask!
